In [1]:
import os
import sys

import numpy as np
import pandas as pd
from datetime import datetime
import pdb
import xarray as xr
import matplotlib.pyplot as plt
import puv

import solo

sys.path.append(r'c:\checkouts\python\TUD-COASTAL\python')

In [2]:
dataFolder="ps_data/atm_pressure_removed/"

ds1 = xr.open_dataset(dataFolder+"ps1.nc")
ds2 = xr.open_dataset(dataFolder+"ps2.nc")
ds3 = xr.open_dataset(dataFolder+"ps3.nc")
ds4 = xr.open_dataset(dataFolder+"ps4.nc")
ds5 = xr.open_dataset(dataFolder+"ps5.nc")
ds6 = xr.open_dataset(dataFolder+"ps6.nc")

In [3]:
ds1 = ds1.dropna(dim='t')
ds2 = ds2.dropna(dim='t')
ds3 = ds3.dropna(dim='t')
ds4 = ds4.dropna(dim='t')
ds5 = ds5.dropna(dim='t')
ds6 = ds6.dropna(dim='t')

In [4]:
# dt = ds1.isel(t=1).t - ds1.isel(t=0).t
# sf = np.timedelta64(1, 's') / dt.values
# ds1['sf'] = sf
# ds2['sf'] = sf
# ds3['sf'] = sf
# ds4['sf'] = sf

# dt = ds5.isel(t=1).t - ds5.isel(t=0).t
# sf = np.timedelta64(1, 's') / dt.values
# ds5['sf'] = sf
# ds6['sf'] = sf

In [5]:
fs1234 = ds1.sf.values
fs56   = ds5.sf.values

rho_sea = 1035  # kg/m3
g = 9.81  # m/s2

ds1['h'] = (ds1['p']/rho_sea/g + ds1['zi'] - ds1['zb']).mean(dim='N') #* 1.1
ds2['h'] = (ds2['p']/rho_sea/g + ds2['zi'] - ds2['zb']).mean(dim='N') #* 1.1
ds3['h'] = (ds3['p']/rho_sea/g + ds3['zi'] - ds3['zb']).mean(dim='N') #* 1.1
ds4['h'] = (ds4['p']/rho_sea/g + ds4['zi'] - ds4['zb']).mean(dim='N') #* 1.1
ds5['h'] = (ds5['p']/rho_sea/g + ds5['zi'] - ds5['zb']).mean(dim='N') #* 1.1
ds6['h'] = (ds6['p']/rho_sea/g + ds6['zi'] - ds6['zb']).mean(dim='N') #* 1.1

ds1['zi'] = ds1['zi'].mean(dim='N')
ds1['zb'] = ds1['zb'].mean(dim='N')

ds2['zi'] = ds2['zi'].mean(dim='N')
ds2['zb'] = ds2['zb'].mean(dim='N')

ds3['zi'] = ds3['zi'].mean(dim='N')
ds3['zb'] = ds3['zb'].mean(dim='N')

ds4['zi'] = ds4['zi'].mean(dim='N')
ds4['zb'] = ds4['zb'].mean(dim='N')

ds5['zi'] = ds5['zi'].mean(dim='N')
ds5['zb'] = ds5['zb'].mean(dim='N')

ds6['zi'] = ds6['zi'].mean(dim='N')
ds6['zb'] = ds6['zb'].mean(dim='N')

ds1['h'].attrs = {'long_name': 'mean water level', 'units': 'm+NAP'}
ds2['h'].attrs = {'long_name': 'mean water level', 'units': 'm+NAP'}
ds3['h'].attrs = {'long_name': 'mean water level', 'units': 'm+NAP'}
ds4['h'].attrs = {'long_name': 'mean water level', 'units': 'm+NAP'}
ds5['h'].attrs = {'long_name': 'mean water level', 'units': 'm+NAP'}
ds6['h'].attrs = {'long_name': 'mean water level', 'units': 'm+NAP'}

display(ds6)

<xarray.Dataset>
Dimensions:  (t: 890, N: 9600)
Coordinates:
  * t        (t) datetime64[ns] 2023-09-23T11:00:00 ... 2023-09-29T15:10:00
  * N        (N) float64 0.0 0.0625 0.125 0.1875 ... 599.8 599.8 599.9 599.9
Data variables:
    p        (t, N) float64 115.9 113.9 118.5 117.9 ... 16.15 14.21 14.21 14.21
    zi       (t) float64 -1.05 -1.05 -1.05 -1.05 ... -1.05 -1.05 -1.05 -1.05
    zb       (t) float64 -1.35 -1.35 -1.35 -1.35 -1.35 ... -1.4 -1.4 -1.4 -1.4
    sf       float64 ...
    name     object ...
    h        (t) float64 0.3126 0.3139 0.3149 0.3157 ... 0.3545 0.3515 0.3516
Attributes: (12/16)
    Conventions:               CF-1.6
    name:                      ps6
    instrument:                ps6
    instrument type:           Ruskin RBR Solo
    instrument serial number:  208682
    epsg:                      28992
    ...                        ...
    summary:                   Hydraulic fieldwork pressure data, with pressu...
    contact person:            Marlies van der Lugt
    emailadres:                m.a.vanderlugt@tudelft.nl
    construction datetime:     10-Nov-2023 (15:14:23)
    version:                   v1
    version comments:          constructed with xarray

In [6]:
# p1 = ds1.p.mean(dim='N')
# p2 = ds2.p.mean(dim='N')
# p3 = ds3.p.mean(dim='N')
# p4 = ds4.p.mean(dim='N')
# p5 = ds5.p.mean(dim='N')
# p6 = ds6.p.mean(dim='N')

In [7]:
def velocity_signal(ds, near_bed_distance=0.25):
    """
    Calculates velocity signal, which is automatically added to the dataset, and adds relevant attributes.
    Orbital velocity is calculated at z = near_bed_distance (above the bed). Ruessink et al (2012) uses near-bed 
    orbital velocities, which is here assumed to be at 0.01m above bed.
    """
    
    
    # %% do several wave statistics computations, only based on pressure
    ufunc = lambda x, h, zi, zb: puv.pressure2velocity(
        ds.sf.values, x, near_bed_distance, h,
        zi,
        zb,
        rho=rho_sea, g=g)

    ds['eta'], ds['uf_z'], ds['pt'] = xr.apply_ufunc(ufunc,
                                  ds['p'], ds['h'], ds['zi'], ds['zb'],
                                  input_core_dims=[['N'], [], [], []],
                                  output_core_dims=[['N'], ['N'], ['N']],
                                  vectorize=True)
    
    ds['eta'].attrs = {'units': 'm', 'long_name': 'RECONSTRUCTED SURFACE ELEVATION, UNREFERENCED'}
    ds['uf_z'].attrs = {'units': 'm/s', 'long_name': 'RECONSTRUCTED VELOCITY AT z='}
    ds['pt'].attrs = {'units': 'Pa', 'long_name': 'RECONSTRUCTED PRESSURE SIGNAL', 'comment':'FOR CHECKING PURPOSES'}

velocity_signal(ds1)
velocity_signal(ds2)
velocity_signal(ds3)
velocity_signal(ds4)
velocity_signal(ds5)
velocity_signal(ds6)

# pt1 = ds1.pt.mean(dim='N')
# pt2 = ds2.pt.mean(dim='N')
# pt3 = ds3.pt.mean(dim='N')
# pt4 = ds4.pt.mean(dim='N')
# pt5 = ds5.pt.mean(dim='N')
# pt6 = ds6.pt.mean(dim='N')

In [8]:
ds1 = ds1.dropna(dim='t')
ds2 = ds2.dropna(dim='t')
ds3 = ds3.dropna(dim='t')
ds4 = ds4.dropna(dim='t')
ds5 = ds5.dropna(dim='t')
ds6 = ds6.dropna(dim='t') 

In [9]:
def velocity_skewness_asymmetry(ds):
    """
    Calculates velocity skewness/asymmetry from velocity signal. 
    """
    ufunc = lambda p: puv.compute_SkAs(
        ds.sf.values, p)

    ds['vel_Sk'], ds['vel_As'], ds['vel_sig'] = xr.apply_ufunc(ufunc,
                                  ds['uf_z'],
                                  input_core_dims=['N'],
                                  output_core_dims=[[], [], []],
                                  vectorize=True)
    ds['vel_Sk'] = ds['vel_Sk'] / ds['vel_sig']**3
    ds['vel_As'] = ds['vel_As'] / ds['vel_sig']**3
    return

velocity_skewness_asymmetry(ds1)
velocity_skewness_asymmetry(ds2)
velocity_skewness_asymmetry(ds3)
velocity_skewness_asymmetry(ds4)
velocity_skewness_asymmetry(ds5)
velocity_skewness_asymmetry(ds6)

In [10]:
display(ds1)

<xarray.Dataset>
Dimensions:  (t: 894, N: 4800)
Coordinates:
  * t        (t) datetime64[ns] 2023-09-23T10:09:32 ... 2023-09-29T14:59:32
  * N        (N) float64 0.0 0.125 0.25 0.375 0.5 ... 599.5 599.6 599.8 599.9
Data variables:
    p        (t, N) float64 226.5 232.9 236.1 236.4 ... 117.8 117.4 117.4 117.4
    zi       (t) float64 0.377 0.377 0.377 0.377 ... 0.377 0.377 0.377 0.377
    zb       (t) float64 0.227 0.227 0.227 0.227 ... 0.247 0.247 0.247 0.247
    sf       float64 8.0
    name     object ...
    h        (t) float64 0.216 0.2126 0.2081 0.2004 ... 0.1401 0.1406 0.1412
    eta      (t, N) float64 -0.003957 -0.004159 ... -3.306e-05 -2.965e-05
    uf_z     (t, N) float64 -0.02666 -0.02802 -0.02939 ... -0.0002759 -0.0002474
    pt       (t, N) float64 -0.003957 -0.00416 ... -3.306e-05 -2.965e-05
    vel_Sk   (t) float64 -1.278 0.1781 0.9704 -0.8903 ... -0.5946 1.117 0.498
    vel_As   (t) float64 -1.099 1.584 -0.7776 -0.6915 ... 0.2447 0.3795 0.02183
    vel_sig  (t) float64 0.03645 0.001502 0.001529 ... 0.002556 0.001319
Attributes: (12/16)
    Conventions:               CF-1.6
    name:                      ps1
    instrument:                ps1
    instrument type:           Ruskin RBR Solo
    instrument serial number:  202438
    epsg:                      28992
    ...                        ...
    summary:                   Hydraulic fieldwork pressure data, with pressu...
    contact person:            Marlies van der Lugt
    emailadres:                m.a.vanderlugt@tudelft.nl
    construction datetime:     10-Nov-2023 (15:13:47)
    version:                   v1
    version comments:          constructed with xarray

In [11]:
def velocity_Ursell(ds0):
    
    rho_seawater = 1035  # kg/m3
    g = 9.81  # m/s2
    fresolution = 0.03125
    
    ds0 = ds0.dropna(dim='t')
    
    # make a new dataset that has an extra dimension to accomodate for the frequency axis
    ds = xr.Dataset(data_vars={},
                     coords={'t': ds0.t.values,
                             'N': ds0.N.values,
                             'f': np.arange(0, ds0.sf.values / 2, fresolution)})
    ds['f'].attrs = {'units': 'Hz'}
    ds.attrs = ds0.attrs

    # put all variables in this new dataset
    for key in ds0.data_vars:
        ds[key] = ds0[key]

    # extract sampling frequency as explicit variable
    sf = ds.f.values
    
    # compute water depth
#     ds['h'] = (ds['p']/rho_seawater/g + ds['zi'] - ds['zb']).mean(dim='N')
#     ds['h'].attrs = {'long_name': 'mean water level', 'units': 'm+NAP'}
    
#     ds['zi'] = ds['zi'].mean(dim='N')
#     ds['zb'] = ds['zb'].mean(dim='N')
    
    # %% do several wave statistics computations, only based on pressure
    ufunc = lambda x, h, zi, zb: puv.attenuation_corrected_wave_spectrum(
        'horizontal',
        ds.sf.values, x, h,
        zi,
        zb,
        fresolution=fresolution)

    fx, ds['vy'] = xr.apply_ufunc(ufunc,
                                  ds['uf_z'], ds['h'], ds['zi'], ds['zb'],
                                  input_core_dims=[['N'], [], [], []],
                                  output_core_dims=[['f'], ['f']],
                                  vectorize=True)
    ds['vy'].attrs = {'units': 'm2/Hz', 'long_name': 'spectral density'}

    ufunc = lambda vy: puv.get_peak_frequency(ds.f.values, vy)
    ds['fp'] = xr.apply_ufunc(ufunc,
                              ds['vy'],
                              input_core_dims=[['f']],
                              output_core_dims=[[]],
                              vectorize=True)

    ufunc = lambda vy, fp: puv.compute_wave_params(ds.f.values, vy, fmin=0.5 * fp, fmax=5)
    ds0['Hm0'], ds0['Tp'], ds0['Tm01'], ds0['Tm02'], ds0['Tmm10'], ds0['Tps'] = xr.apply_ufunc(ufunc,
                                                                              ds['vy'], ds['fp'],
                                                                              input_core_dims=[['f'], []],
                                                                              output_core_dims=[[], [], [], [], [], []],
                                                                              vectorize=True)
    ds0['Hm0'].attrs = {'units': 'm', 'long_name': 'significant wave height',
                       'computation': 'computed between fmin=0.5fp and fmax=5'}
    ds0['Tp'].attrs = {'units': 's', 'long_name': 'peak wave period',
                      'computation': 'computed between fmin=0.5fp and fmax=5'}
    ds0['Tm01'].attrs = {'units': 's', 'long_name': 'mean wave period',
                        'computation': 'computed between fmin=0.5fp and fmax=5'}
    ds0['Tm02'].attrs = {'units': 's', 'long_name': 'mean wave period',
                        'computation': 'computed between fmin=0.5fp and fmax=5'}
    ds0['Tmm10'].attrs = {'units': 's', 'long_name': 'Tm-1,0',
                         'computation': 'computed between fmin=0.5fp and fmax=5'}
    ds0['Tps'].attrs = {'units': 's', 'long_name': 'peak wave period',
                       'computation':'computed between fmin=0.5fp and fmax=5', 
                       'comment':'smoothed estimate from the discrete spectrum'}
    
    ds0 = ds0.assign({'u_k':('t', puv.disper(w=2*np.pi/ds0['Tmm10'], h=ds0['h'], g=9.81))})
    ds0['u_Ur'] = 3/8 * ds0['Hm0'] * ds0['u_k'] / (ds0['u_k']*ds0['h'])**3
    
    ds0['u_k'].attrs = {'units': '1/m', 'long_name': 'wave number',
                       'computation': 'computation based on dispersion relation with Tm-1,0',
                         'comment': 'calculated from velocity data'}
    ds0['u_Ur'].attrs = {'units': '-', 'long_name': 'Ursell number',
                       'computation': 'computation based on dispersion relation with Tm-1,0',
                         'comment': 'calculated from velocity data'}
    return ds0

ds1 = velocity_Ursell(ds1)
ds2 = velocity_Ursell(ds2)
ds3 = velocity_Ursell(ds3)
ds4 = velocity_Ursell(ds4)
ds5 = velocity_Ursell(ds5)
ds6 = velocity_Ursell(ds6)

C:\Users\khdeb\Documents\Mijn Documenten\School\Universiteit\Civiele Techniek\MSc\Year 2\Q1\Hydraulics Fieldwork\Fieldwork Directory\pressureProcessing\puv.py:348: RuntimeWarning: overflow encountered in sinh
  Sw = 1/w*np.sinh(k*h)/np.cosh(k*elev)


In [12]:
display(ds1)

<xarray.Dataset>
Dimensions:  (t: 894, N: 4800)
Coordinates:
  * t        (t) datetime64[ns] 2023-09-23T10:09:32 ... 2023-09-29T14:59:32
  * N        (N) float64 0.0 0.125 0.25 0.375 0.5 ... 599.5 599.6 599.8 599.9
Data variables: (12/20)
    p        (t, N) float64 226.5 232.9 236.1 236.4 ... 117.8 117.4 117.4 117.4
    zi       (t) float64 0.377 0.377 0.377 0.377 ... 0.377 0.377 0.377 0.377
    zb       (t) float64 0.227 0.227 0.227 0.227 ... 0.247 0.247 0.247 0.247
    sf       float64 8.0
    name     <U3 'ps1'
    h        (t) float64 0.216 0.2126 0.2081 0.2004 ... 0.1401 0.1406 0.1412
    ...       ...
    Tm01     (t) float64 11.08 14.22 14.92 12.14 ... 13.13 14.06 14.18 14.56
    Tm02     (t) float64 6.872 10.68 12.21 7.892 7.369 ... 9.15 10.54 10.77 11.5
    Tmm10    (t) float64 14.83 15.67 15.81 15.16 ... 15.4 15.62 15.65 15.73
    Tps      (t) float64 16.0 16.0 16.0 16.0 16.0 ... 16.0 16.0 16.0 16.0 16.0
    u_k      (t) float64 0.2912 0.2778 0.2784 0.2958 ... 0.3433 0.3421 0.3396
    u_Ur     (t) float64 0.4345 0.04113 0.07394 0.128 ... 0.1045 0.1363 0.08455
Attributes: (12/16)
    Conventions:               CF-1.6
    name:                      ps1
    instrument:                ps1
    instrument type:           Ruskin RBR Solo
    instrument serial number:  202438
    epsg:                      28992
    ...                        ...
    summary:                   Hydraulic fieldwork pressure data, with pressu...
    contact person:            Marlies van der Lugt
    emailadres:                m.a.vanderlugt@tudelft.nl
    construction datetime:     10-Nov-2023 (15:13:47)
    version:                   v1
    version comments:          constructed with xarray

In [13]:
dataFolder="ps_data/tailored_with_ps_UrSkAs/"

ds1_old = xr.open_dataset(dataFolder+"ps1.nc")
ds2_old = xr.open_dataset(dataFolder+"ps2.nc")
ds3_old = xr.open_dataset(dataFolder+"ps3.nc")
ds4_old = xr.open_dataset(dataFolder+"ps4.nc")
ds5_old = xr.open_dataset(dataFolder+"ps5.nc")
ds6_old = xr.open_dataset(dataFolder+"ps6.nc")

def add_velocity_k_Ur_Sk_and_As_to_tailored_data(ds, ds_old):
    """
    Adds calculated velocity skewness/asymmetry to tailored dataset
    and adds it to dataset (including attributes)
    """
    ds_old['vel_Sk'] = ds['vel_Sk']
    ds_old['vel_As'] = ds['vel_As']
    ds_old['vel_sig'] = ds['vel_sig']
    ds_old['u_k'] = ds['u_k']
    ds_old['u_Ur'] = ds['u_Ur']
    
    ds_old['vel_Sk'].attrs = {'units': '-', 'long_name': 'velocity skewness (at z = mean water level)'}
    ds_old['vel_As'].attrs = {'units': '-', 'long_name': 'velocity asymmetry (at z = mean water level)'}
    ds_old['vel_sig'].attrs = {'units': 'm/s', 'long_name': 'STD'}
    ds_old['u_k'].attrs = {'units': '1/m', 'long_name': 'wave number',
                       'computation': 'computation based on dispersion relation with Tm-1,0',
                         'comment': 'calculated from velocity data'}
    ds_old['u_Ur'].attrs = {'units': '-', 'long_name': 'Ursell number',
                       'computation': 'computation based on dispersion relation with Tm-1,0',
                         'comment': 'calculated from velocity data'}
    
    return ds_old

ds1 = add_velocity_k_Ur_Sk_and_As_to_tailored_data(ds1, ds1_old)
ds2 = add_velocity_k_Ur_Sk_and_As_to_tailored_data(ds2, ds2_old)
ds3 = add_velocity_k_Ur_Sk_and_As_to_tailored_data(ds3, ds3_old)
ds4 = add_velocity_k_Ur_Sk_and_As_to_tailored_data(ds4, ds4_old)
ds5 = add_velocity_k_Ur_Sk_and_As_to_tailored_data(ds5, ds5_old)
ds6 = add_velocity_k_Ur_Sk_and_As_to_tailored_data(ds6, ds6_old)

In [14]:
ncOutFile = "ps_data/tailored_with_ps_and_vel_UrSkAs/"

ds1.to_netcdf(ncOutFile + f'{ds1.instrument}' + '.nc')
ds2.to_netcdf(ncOutFile + f'{ds2.instrument}' + '.nc')
ds3.to_netcdf(ncOutFile + f'{ds3.instrument}' + '.nc')
ds4.to_netcdf(ncOutFile + f'{ds4.instrument}' + '.nc')
ds5.to_netcdf(ncOutFile + f'{ds5.instrument}' + '.nc')
ds6.to_netcdf(ncOutFile + f'{ds6.instrument}' + '.nc')

In [15]:
# fig, ax = plt.subplots(6, 1, figsize=(15, 18))

# ax[0].plot(ds1.t, p1)
# ax[0].plot(ds1.t, pt1)

# ax[1].plot(ds2.t, p2)
# ax[1].plot(ds2.t, pt2)

# ax[2].plot(ds3.t, p3)
# ax[2].plot(ds3.t, pt3)

# ax[3].plot(ds4.t, p4)
# ax[3].plot(ds4.t, pt4)

# ax[4].plot(ds5.t, p5)
# ax[4].plot(ds5.t, pt5)

# ax[5].plot(ds6.t, p6)
# ax[5].plot(ds6.t, pt6)